# Random Environment with Four Stages Repairmen Model

This example demonstrates a random environment model with four different operational stages.

## Model Features:
- **Environment**: 4 stages (UP, DOWN, FAST, SLOW)
- **Queueing Network**: 2 stations (Delay + PS Queue) per stage
- **Job Population**: 30 jobs
- **Dynamic Rates**: Service rates change based on environment stage
- **Stage Transitions**: Markovian transitions between environment stages

## Environment Stages:
1. **Stage1 (UP)**: Rate[0]=4, Rate[1]=1
2. **Stage2 (DOWN)**: Rate[0]=3, Rate[1]=2  
3. **Stage3 (FAST)**: Rate[0]=2, Rate[1]=3
4. **Stage4 (SLOW)**: Rate[0]=1, Rate[1]=4

The metasolver analyzes performance across all stages considering stage transition dynamics.

In [ ]:
// Kotlin notebook
import jline.*
import jline.lang.*
import jline.lang.nodes.*
import jline.lang.processes.*
import jline.lang.constant.*
import jline.solvers.env.*
import jline.solvers.fluid.*
import jline.util.matrix.*

In [ ]:
// Helper function to generate a queueing network for random environment
fun renvGenqn(rates: DoubleArray, N: Int): Network {
    val qnet = Network("qn1")
    
    val node1 = Delay(qnet, "Queue1")
    val node2 = Queue(qnet, "Queue2", SchedStrategy.PS)
    
    val jobclass = ClosedClass(qnet, "Class1", N, node1, 0)
    
    node1.setService(jobclass, Exp(rates[0]))
    node2.setService(jobclass, Exp(rates[1]))
    
    val P = qnet.initRoutingMatrix()
    P.set(jobclass, jobclass, node1, node2, 1.0)
    P.set(jobclass, jobclass, node2, node1, 1.0)
    qnet.link(P)
    
    return qnet
}

In [ ]:
// Model parameters
val N = 30  // Job population
val M = 2   // Number of stations per submodel
val E = 4   // Number of environment stages

// Create environment model
val envModel = Env("MyEnv", E)
val envName = arrayOf("Stage1", "Stage2", "Stage3", "Stage4")
val envType = arrayOf("UP", "DOWN", "FAST", "SLOW")

// Create rate matrix
// rate[station][stage] = service rate for station in given stage
val rates = Array(E) { stage ->
    doubleArrayOf(
        (E - stage).toDouble(),  // First station: 4,3,2,1
        (stage + 1).toDouble()   // Second station: 1,2,3,4
    )
}

println("Rate matrix:")
rates.forEachIndexed { stage, stageRates ->
    println("Stage $stage: ${stageRates.contentToString()}")
}

In [ ]:
// Create queueing networks for each environment stage
val envSubModels = Array(E) { stage ->
    renvGenqn(rates[stage], N)
}

// Add stages to environment model
for (e in 0 until E) {
    envModel.addStage(e, envName[e], envType[e], envSubModels[e])
}

In [ ]:
// Define environment transition rates
val envRates = arrayOf(
    doubleArrayOf(0.0, 0.5, 0.0, 0.0),
    doubleArrayOf(0.0, 0.0, 0.5, 0.5),
    doubleArrayOf(0.5, 0.0, 0.0, 0.5),
    doubleArrayOf(0.5, 0.5, 0.0, 0.0)
)

println("\nEnvironment transition rates:")
envRates.forEachIndexed { from, transitions ->
    println("From Stage$from: ${transitions.contentToString()}")
}

// Add transitions with Erlang distributions (approximating APH)
for (e in 0 until E) {
    for (h in 0 until E) {
        if (envRates[e][h] > 0) {
            val meanTime = 1.0 / envRates[e][h]
            // Use Erlang distribution of order 2 as approximation for APH
            envModel.addTransition(e, h, Erlang.fitMeanAndOrder(meanTime, 2))
        }
    }
}

In [ ]:
// Create solvers for each submodel
val solvers = Array(E) { stage ->
    FLD(envSubModels[stage])
}

// Create environment solver
val envSolver = ENV(envModel, solvers.toList())

println("\n=== Solving Random Environment Model ===")
println("The metasolver considers an environment with $E stages and queueing networks with $M stations.")
println("Every time the stage changes, the queueing network modifies the service rates of the stations.")

In [ ]:
// Get results
try {
    println("\n=== Environment Solution Results ===")
    
    // Get ensemble average
    val avgTable = envSolver.ensembleAvg
    println("Average performance metrics:")
    avgTable.print()
    
} catch (e: Exception) {
    println("Error during solving: ${e.message}")
    println("Note: Some environment solver features may require specific solver configurations")
}

In [ ]:
// Try to get ensemble average tables if available
try {
    val ensembleAvgTables = envSolver.ensembleAvgTables
    println("\n=== Ensemble Average Tables ===")
    ensembleAvgTables.forEachIndexed { index, table ->
        println("\nStage $index (${envName[index]} - ${envType[index]}):")
        table.print()
    }
} catch (e: Exception) {
    println("\nEnsemble average tables not available: ${e.message}")
}